In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable as var
from torch.nn import functional as F
import torchtext.vocab as vocab
import torch.autograd as autograd
from tqdm import tqdm
from pprint import pprint
import json
import _pickle as pkl
# from helpers.utils import StandardNLL
%matplotlib inline

In [2]:
glove = vocab.GloVe(name='6B', dim=50)
# MAX_QUES_LEN = 92
# MAX_CONTEXT_LEN= 921

.vector_cache/glove.6B.zip: 862MB [04:31, 3.18MB/s]                               
100%|██████████| 400000/400000 [00:05<00:00, 68116.00it/s]


In [10]:
with open('../data/data.json', 'r') as f:
    data = json.load(f)

In [11]:
data['X_train'][:2]

[['The clergy was divided into two types: the secular clergy, who lived out in the world, and the regular clergy, who lived under a religious rule and were usually monks. Throughout the period monks remained a very small proportion of the population, usually less than one per cent. Most of the regular clergy were drawn from the nobility, the same social class that served as the recruiting ground for the upper levels of the secular clergy. The local parish priests were often drawn from the peasant class. Townsmen were in a somewhat unusual position, as they did not fit into the traditional three-fold division of society into nobles, clergy, and peasants. During the 12th and 13th centuries, the ranks of the townsmen expanded greatly as existing towns grew and new population centres were founded. But throughout the Middle Ages the population of the towns probably never exceeded 10 per cent of the total population.',
  'What percentage of the European population consisted of monks?',
  'le

In [12]:
print(glove.vectors.size())

torch.Size([400000, 50])


In [13]:
MAX_CONTEXT_LEN = 0
MAX_QUES_LEN = 0
# print(data['X_train'][0])
for (c,q, _) in data['X_train'][:8000]:
#     print(len(c.split()))
    if len(c.split())> MAX_CONTEXT_LEN:
        MAX_CONTEXT_LEN= len(c)
    if len(q.split())> MAX_QUES_LEN:
        MAX_QUES_LEN= len(q)
print(MAX_QUES_LEN)
print(MAX_CONTEXT_LEN)

62
921


In [14]:


def clean(token):
    cleaned_token = token.strip(".,?!-:;'()[]\"`")
    if cleaned_token[-2:] == "'s":
        cleaned_token = cleaned_token[:-2]
    if cleaned_token[-2:] == "'t":
        cleaned_token = cleaned_token[:-2]+'t'
    return cleaned_token

def vectorize(input_txt, max_len):
    input_seq = [clean(w) for w in input_txt.split(" ") if len(clean(w).strip())]
    glove_vec = []
    for w in input_seq:
        try:
            glove_vec.append(glove.stoi[w])
        except:
            glove_vec.append(400001) # <unk> token
    if len(glove_vec)<max_len:
        padding_zeros = [400000]*(max_len-len(glove_vec)) # <pad> token
        glove_vec = padding_zeros + glove_vec
    return glove_vec[:max_len]
    
def make_data(raw_X):
    X = []
    y = []
    for (c, q, a) in raw_X:
        context_rep = vectorize(c.lower(), MAX_CONTEXT_LEN)
        ques_rep = vectorize(q.lower(), MAX_QUES_LEN)
        X.append(context_rep+ques_rep) #only context for now
    return X

DIM=50
glove = vocab.GloVe(name='6B', dim=DIM)

glove.stoi['<pad>'] = len(glove.stoi)+1
glove.vectors = torch.cat((glove.vectors, torch.zeros(1, DIM)))
glove.stoi['<unk>'] = len(glove.stoi)+1 # add token->index for unknown/oov
glove.vectors = torch.cat((glove.vectors, torch.ones(1, DIM)*-1)) # add index->vec for unknown/oov

print(glove.vectors.size())
VOCAB_SIZE = glove.vectors.size()[0]

torch.Size([400002, 50])


In [15]:
idx = 5
example_X = (data['X_train'][idx])
example_y = (data['y_train'][idx])
print("Context:", example_X[0])
print("Question:", example_X[1])
print("Answer Span:", example_y)
print("Answer:", example_X[0][example_y[0]:example_y[1]])
X = vectorize(example_X[0].lower(), MAX_CONTEXT_LEN)

Context: The boreholes on Funafuti, at the site now called Darwin's Drill, are the result of drilling conducted by the Royal Society of London for the purpose of investigating the formation of coral reefs to determine whether traces of shallow water organisms could be found at depth in the coral of Pacific atolls. This investigation followed the work on The Structure and Distribution of Coral Reefs conducted by Charles Darwin in the Pacific. Drilling occurred in 1896, 1897 and 1898. Professor Edgeworth David of the University of Sydney was a member of the 1896 "Funafuti Coral Reef Boring Expedition of the Royal Society", under Professor William Sollas and lead the expedition in 1897. Photographers on these trips recorded people, communities, and scenes at Funafuti.
Question: What was Darwin's work on coral reefs titled?
Answer Span: [347, 392]
Answer: The Structure and Distribution of Coral Reefs


In [16]:
y_start = []
y_end = []

for (c,q,ans) in data['X_train']:
#     temp_s = list(0 for i in range(MAX_CONTEXT_LEN))
#     temp_e = list(0 for i in range(MAX_CONTEXT_LEN))
    temp_c = c.split()
    temp_c = ['PAD']*(MAX_CONTEXT_LEN-len(temp_c))+temp_c
    con_new = ' '.join(temp_c)
    start_pos = len(con_new[:con_new.find(ans)].split())-1
    end_pos = start_pos+(len(ans.split()))-2
#     if(start_pos)==921 or (end_pos)==921:
#         print(c,ans)
#         print(start_pos)
#         print(end_pos)
#     if(start_pos)==1:
#         print(c,ans)
#         print(start_pos)
#         print(end_pos)
#     print(temp_c)
#     print(c)
#     print(ans)
#     print(start_pos, end_pos)
#     temp_s[start_pos]=1
#     y_start.append(temp_s)
#     temp_e[end_pos]=1
#     y_end.append(temp_e)
    y_start.append(start_pos)
    y_end.append(end_pos)
#     print(y_start, y_end)

In [17]:
X_pass = make_data(data['X_train'][:8000])
# y_pass = data['y_train'][:400]
y_pass_start = y_start[:8000]
y_pass_end = y_end[:8000]
print(len(X_pass))
print(len(y_pass_start))
print(len(y_pass_end))

8000
8000
8000


In [18]:
class ModelV1(nn.Module):
    def __init__(self, config):
        super(ModelV1, self).__init__()
        
        self.input_size = config.get("input_size", MAX_CONTEXT_LEN+MAX_QUES_LEN)
        self.hidden_size = config.get("hidden_size", 128)
        self.output_size = config.get("output_size", MAX_CONTEXT_LEN)
        self.n_layers = config.get("n_layers", 1)
        self.vocab_size = config.get("vocab", VOCAB_SIZE)
        self.emb_dim = config.get("embedding_dim", DIM)
        self.bidir = config.get("Bidirectional", True)
        self.dirs = int(self.bidir)+1
        self.lr = config.get("learning_rate", 1e-3)
        self.batch_size = config.get("batch_size", 1)
        self.epochs = config.get("epochs", 5)
        self.opt = config.get("opt", "SGD")
        
        if self.opt == 'Adam':
            self.opt = optim.Adam
        else:
            self.opt = optim.SGD
        
        self.encoder = nn.Embedding(self.vocab_size, self.emb_dim)
        self.lstm = nn.LSTM(self.emb_dim, self.hidden_size, self.n_layers, bidirectional=self.bidir)
        self.decoder_start = nn.Linear(self.hidden_size, self.output_size)
        self.decoder_end = nn.Linear(self.hidden_size, self.output_size)
        self.init_weights()
    
    def init_weights(self):
        weight_scale = 0.01
        self.encoder.weight.data = glove.vectors
        self.decoder_start.bias.data.fill_(0)
        self.decoder_start.weight.data.uniform_(-weight_scale, weight_scale)
        self.decoder_end.bias.data.fill_(0)
        self.decoder_end.weight.data.uniform_(-weight_scale, weight_scale)

    def init_hidden(self, bs=None):
        if bs is None:
            bs = self.batch_size
        weight = next(self.parameters()).data
        return var(weight.new(self.n_layers*self.dirs, bs, self.hidden_size).zero_())
        
    def forward(self, inputs):
        if len(inputs)==1:
            inputs = var(torch.LongTensor(inputs[0]))
        else:
            inputs = var(torch.LongTensor(inputs))
#         print(inputs.size())
        embeds = self.encoder(inputs).permute(1,0,2)# get glove repr
#         print("embeds:", embeds.size())
        seq_len = embeds.size()[0]
        lstm_op, self.hidden = self.lstm(embeds, self.hidden)
        # print("lstm op:", lstm_op.size()) # (seq_len, bs, hidden_size*(dirs=2 for bi))
        lstm_op = lstm_op.permute(1, 0, 2) # (seq_len, bs, hdim)->(bs, seq_len, hdim)
#         print(lstm_op)
        
        end_pred = lstm_op[:, -1, :self.hidden_size] # forward direction
        start_pred = lstm_op[:, -1, self.hidden_size:] # reverse direction
        
        # print("lstm start, end preds:", start_pred.size(), end_pred.size())
        out_start = F.log_softmax(self.decoder_start(start_pred), dim=-1)
        out_end = F.log_softmax(self.decoder_end(end_pred), dim=-1)
        # print("outs:", out_start.size(), out_end.size())
        out = torch.cat((out_start, out_end), 1)
#         print("out:", out.size())
        return out
    
    def fit(self, X, y_s, y_e):
        opt = self.opt(self.parameters(), self.lr)
        losses = [] # epoch loss
        for epoch in range(self.epochs):
            print("epoch:", epoch)
            bs = self.batch_size
#             print(bs)
            bloss = 0.0 # batch loss
#             loss_epoch = []
#             print(len(y_s))
            for i in range(0, len(y_s)-bs+1, bs):
#                 print(i)
#             for (x_data, y_data_s, y_data_e) in zip(X, y_s, y_e):
                #print("batch:", bindex)
                h, c = self.init_hidden(), self.init_hidden()
                self.hidden = (h, c)
                # print(h.size(), c.size())
                opt.zero_grad()
#                 Xb = X[i:i+bs]
#                 Xb = torch.LongTensor(Xb)
#                 # print("Xb:", Xb.size())
#                 yb = var(torch.LongTensor(y[i:i+bs]))
#                 # print("yb:", yb.size())
#                 pred = self.forward(Xb) #prediction on batch features
                
                x = X[i:i+bs]
#                 print(x)
#                 y = autograd.Variable(torch.LongTensor([y_data_s, y_data_e]), requires_grad=False)
    #             Xb = X
    #             yb = var(torch.LongTensor(y[i:i+bs]))
                pred = self.forward(x) #prediction on batch features
#                 print(pred)
#                 print(autograd.Variable(torch.LongTensor(y_s[i:i+bs])))
                y_s_temp = torch.LongTensor(y_s[i:i+bs])
                y_e_temp = torch.LongTensor(y_e[i:i+bs])
#                 print(y_s_temp.size())
#                 print(y_s_temp.size())
#                 print(pred.size())
#                 print(y_data_s)
            
#                 loss= criterion(y_predicted, y)
                loss = F.nll_loss(pred[:, :self.output_size], autograd.Variable(y_s_temp)) \
                     + F.nll_loss(pred[:, self.output_size:], autograd.Variable(y_e_temp)) 
#                 loss_epoch.append(loss.data[0])
                bloss += loss.data[0]/bs

                loss.backward()
                opt.step()
            losses.append(bloss)
            print(losses[-1], end=', change: ')
            if len(losses)>1:
                diff = losses[-2]-losses[-1]
                rel_diff = diff/losses[-2]
                print("%s"%rel_diff, "%")
            else:
                print("00.0%")
        return losses

    def predict(self, X, bs=None):
        self.hidden = (self.init_hidden(bs), self.init_hidden(bs))
        result = self.forward(X)
        return self.get_span_indices(result)
    
    def get_span_indices(self, preds):
        s_pred = preds[:, :self.output_size]
        e_pred = preds[:, self.output_size:]
        _,  s_index = torch.max(s_pred, -1)
        _,  e_index = torch.max(e_pred, -1)
        return torch.cat((s_index.unsqueeze(1), e_index.unsqueeze(1)), -1)

In [19]:
conf = {"learning_rate": 0.5, 
        "epochs": 20,
       "batch_size": 50}
model = ModelV1(conf)

In [ ]:
res = model.fit(X_pass, y_pass_start, y_pass_end)

epoch: 0
38.23764572143555, change: 00.0%
epoch: 1
34.134663753509514, change: 0.10730215970451218 %
epoch: 2
33.62588384628299, change: 0.014905080386919468 %
epoch: 3
33.42245462417603, change: 0.006049780670061082 %
epoch: 4
33.28580827713014, change: 0.004088459348136774 %
epoch: 5
33.16873907089234, change: 0.00351709068510826 %
epoch: 6
33.053320121765154, change: 0.0034797508847261758 %
epoch: 7
32.92940980911255, change: 0.0037488007920575407 %
epoch: 8
32.793599472045905, change: 0.004124287008298016 %
epoch: 9
32.64163034439087, change: 0.00463410940249418 %
epoch: 10


In [ ]:
for x, y in zip(data['X_val'][:200], data['y_val'][:200]):
    c = x[0]
    a = x[2]
    x = make_data([x])
#     print(x)
    temp_c = c.split()
    temp_c = ['PAD']*(MAX_CONTEXT_LEN-len(temp_c))+temp_c
#     con_new = ' '.join(temp_c)
#     print(con_new)
    res = model.predict([x], bs=1).data.tolist()[0]
    print("Predicted span:", res)
    if res[0]>res[1]:
        res[0], res[1] = res[1], res[0]
        print("switched to:", res)
    print("Predicted Answer:", temp_c[res[0]:res[1]])
    print("Actual:", a)
    print("="*50)